In [12]:
import numpy as np
import matplotlib.pyplot as plt
import Circuit
from EC09 import EC09
import pandas as pd
from numpy import random

In [13]:
frequencies = Circuit.dec_step(1e-2, 1e5, 72)

In [14]:
import random
# from openpyxl import Workbook
samples=50000
# Generate random values for three variables
Rs = [random.uniform(15,35) for _ in range(samples)]
R1 = [random.uniform(15,35) for _ in range(samples)]
R2 = [random.uniform(15,35) for _ in range(samples)]
R3 = [random.uniform(15,35) for _ in range(samples)]
CPE_Y1 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n1 = [random.uniform(0.5, 1) for _ in range(samples)]
CPE_Y2 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n2 = [random.uniform(0.5, 1) for _ in range(samples)]
CPE_Y3 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n3 = [random.uniform(0.5, 1) for _ in range(samples)]


In [15]:
output_array_9 = np.stack((Rs,R1,R2,R3, CPE_Y1, CPE_n1,CPE_Y2, CPE_n2,CPE_Y3, CPE_n3), axis=1)

In [16]:
num_rows = len(output_array_9)
num_rows

50000

In [17]:
num_freq = len(output_array_9)
num_rows = 72
num_columns = 3

# Create an empty numpy array to store the frequency response data
frequency_response_array_9= np.empty((num_freq, num_rows, num_columns))
# len(frequency_response_array[0][0])

In [18]:
# As output_array is a 2D NumPy array
for idx in range(output_array_9.shape[0]):
    row = output_array_9[idx]
    b = EC09(*row)
    Z = b.freq_response(frequencies)

    # Store the frequency values  in the first column
    frequency_response_array_9[idx, :, 0] = frequencies

    # Store the Absolute Value of Z in the second column
    frequency_response_array_9[idx, :, 1] = np.abs(Z)

    # Store the Phase values of Z in the third column
    frequency_response_array_9[idx, :, 2] = np.angle(Z)


# Now, frequency_response_array has the shape (50000, 72, 3)



In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assuming frequency_response_array has shape (50000, 72, 3) and output_array has shape (50000, 3)

# Reshape frequency_response_array to have shape (50000, 216)
X = frequency_response_array_9.reshape((frequency_response_array_9.shape[0], -1))

y = output_array_9

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Training R^2 Score:", train_score)
print("Testing R^2 Score:", test_score)


Training R^2 Score: 0.4501852853458934
Testing R^2 Score: 0.4465508876570133


In [20]:
for i in range(1000):    
    random_idx = np.random.randint(0, len(frequency_response_array_9))
    # Select the random element from frequency_response_array
    random_element = frequency_response_array_9[random_idx]

    # Reshape the random element to match the input shape expected by the model
    reshaped_element = random_element.reshape(1, -1)

    # Predict the corresponding output value
    predicted_output = model.predict(reshaped_element)

    # Extract the corresponding actual output value from the output array
    actual_output =output_array_9[random_idx]

    # Print the predicted and actual output values for comparison
    print("Predicted Output:",predicted_output)
    print("Actual Output:",actual_output)


Predicted Output: [[2.88139648e+01 2.58750000e+01 2.43593750e+01 2.38437500e+01
  6.33239746e-04 9.43115234e-01 2.91824341e-04 9.04296875e-01
  4.42504883e-04 7.57812500e-01]]
Actual Output: [2.88360437e+01 2.98493750e+01 2.84813256e+01 1.56770945e+01
 6.54475506e-04 9.06830381e-01 3.19406651e-04 9.24186624e-01
 3.48594829e-04 7.50155695e-01]
Predicted Output: [[3.46523438e+01 2.91250000e+01 2.83750000e+01 2.83125000e+01
  3.89099121e-04 6.81884766e-01 5.34057617e-04 8.28125000e-01
  5.30242920e-04 8.12500000e-01]]
Actual Output: [3.48994737e+01 2.69655671e+01 3.09139479e+01 2.75321786e+01
 5.03003016e-04 5.48374477e-01 4.34511543e-04 8.74576808e-01
 8.72959320e-04 7.96514869e-01]
Predicted Output: [[2.57485352e+01 2.65625000e+01 2.57968750e+01 2.53437500e+01
  4.73022461e-04 7.45361328e-01 3.68118286e-04 7.41210938e-01
  4.69207764e-04 7.18750000e-01]]
Actual Output: [2.56462569e+01 3.20655506e+01 1.71979992e+01 2.86084424e+01
 4.54300661e-04 5.77116767e-01 4.05256690e-04 8.37045201e-

In [21]:
from sklearn.metrics import mean_squared_error,accuracy_score

# Predict outputs for the entire test set
predicted_outputs = model.predict(X_test)

# Compute mean squared error
mse = mean_squared_error(y_test, predicted_outputs)
accuracy=100-mse
# Print the mean squared error
print("Mean Squared Error:", mse)
print("Accuracy:",accuracy)


Mean Squared Error: 6.216999883494571
Accuracy: 93.78300011650543


In [22]:
np.save('frequency_response_array_9.npy',frequency_response_array_9)